In [266]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [267]:
int_reactions= list()
i= -1
n= 0
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()
    n= int((len(csvFile)-1)/2)

    flt= float(csvFile.iloc[n,  2])
    
    if v == 1 and csvFile.iloc[n,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[n,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


[]
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16a4colipa_e', 'EX_doxrbcn_e', 'EX_o2s_e', 'EX_xtsn_e', 'EX_pacald_e', 'EX_zn2_e', '12DGR120tipp', '12DGR140tipp', '12DGR141tipp', '12DGR160tipp', 'EX_pnto__R_e', '12DGR161tipp', '12DGR18

In [268]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

    i3= int(((len(csvFile)-1)/2)+1)

    while i3< len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4< int(((len(csvFile)-1)/2)):
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [269]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
one_to_three_data_point_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat above zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(4)
    
    
    my_series = d['Biomass'].squeeze()


    if (len(d)> 30 or len(d)==30) and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.0001 and d.iloc[n2,  0] < max(d["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.0001 and d.iloc[n2,  0] < max(d["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        if count> 2 and count< len(d)-2:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 6:
            flat_above_zero.append(r_id)
        
        elif count> len(d)-3 and len(d)> 5:
            flat_above_zero.append(r_id)
        
        elif len(d)<4:
            one_to_three_data_point_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['FBA3', 'O2tex']

In [270]:
len(flat_above_zero)+len(diff_indecr_above_zero)+len(indecr_and_stable_above_zero)+len(one_to_three_data_point_above_zero)


640

In [271]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
one_to_three_data_point_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat below zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(4)
    
    my_series = d['Biomass'].squeeze()
    

    if (len(d)> 30 or len(d)==30) and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.0001 and d.iloc[n2,  0] < max(d["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.0001 and d.iloc[n2,  0] < max(d["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        if count> 2 and count< len(d)-2:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 6:
            flat_below_zero.append(r_id)
        
        elif count> len(d)-3 and len(d)> 5:
            flat_below_zero.append(r_id)
        
        elif len(d)<4:
            one_to_three_data_point_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


['EX_o2_e',
 '2DGULRGy',
 '2DGULRx',
 '2DGULRy',
 'BETALDHx',
 'BSORy',
 'DKGLCNR2y',
 'FRUpts2pp',
 'NODOy',
 'MECDPDH2',
 '2DGLCNRy',
 'DOGULND']

In [272]:
len(flat_below_zero)+len(diff_indecr_below_zero)+len(indecr_and_stable_below_zero)+len(one_to_three_data_point_below_zero)


499

In [273]:
i2= 0
mon_incr_above_zero= list()
mon_decr_above_zero= list()

if len(diff_indecr_above_zero)> 0:
    
    for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_above_zero)
set2= set(mon_decr_above_zero)
set3= set(mon_incr_above_zero)
inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_above_zero


['CYSS',
 'DDPA',
 '3OAR140',
 'IPMD',
 'DHQTi',
 'DHAD2',
 'UPPDC1',
 'PRMICI',
 'UAAGDS',
 'SULR',
 'GMPS2',
 'PPND',
 'TMDS',
 'PE160abcpp',
 'ACLS',
 'ADSL1r',
 'K2L4Aabcpp',
 'ARGSL',
 'FCLT',
 'ADSS',
 'NDPK2',
 'HISTP',
 'PPBNGS',
 'IG3PS',
 'ANPRT',
 'UAMAS',
 'PRATPP',
 'USHD',
 'THRS',
 'CS',
 'THRD_L',
 'P5CR',
 '3OAS140',
 'G6PDH2r',
 'AGPAT160',
 'UGMDDS',
 'ANS',
 'PRAIS',
 'HMBS',
 'SHSL1',
 'O2tpp',
 'DAPE',
 'PRAGSr',
 'GLUPRT',
 'OMCDC',
 'PRASCSi',
 'HSTPT',
 '5DOAN',
 'UHGADA',
 'PSCVT',
 'PSD160',
 'THDPS',
 'PRAMPC',
 'LPADSS',
 'UDCPDP',
 'ADSL2r',
 'SO4tex',
 'DHDPRy',
 'ATPPRT',
 'DM_5drib_c',
 'DHAD1',
 'PSP_L',
 'MPTG',
 'ACONTa',
 'ASPK',
 'KDOCT2',
 'ALAR',
 'DAPDC',
 'G5SADs',
 'CHORM',
 'IPPS',
 'UAGPT3',
 'DASYN160',
 'METAT',
 'KDOPS',
 'NNATr',
 'ICDHyr',
 'PSERT',
 'SHKK',
 'HCO3E',
 'NDPK4',
 'PSSA160',
 'UAGAAT',
 'GLNS',
 'CU2tpp',
 'CHORS',
 'PRFGS',
 'UAGDP',
 'MTHFR2',
 'ACCOAC',
 'U23GAAT',
 'PRAIi',
 'MOAT2',
 'GLU5K',
 'SHK3Dr',
 'ALAALAr',
 

In [274]:
w= list()
w2= list()
i2=0

if len(inANDdecr_above_zero)> 0:
    
    while i2 < len(inANDdecr_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2] < max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3]<max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

        i2= i2+1

w


['3OAR140',
 'UPPDC1',
 'PRMICI',
 'UAAGDS',
 'PPND',
 'TMDS',
 'PE160abcpp',
 'K2L4Aabcpp',
 'FCLT',
 'HISTP',
 'PPBNGS',
 'IG3PS',
 'ANPRT',
 'UAMAS',
 'PRATPP',
 'USHD',
 '3OAS140',
 'AGPAT160',
 'UGMDDS',
 'ANS',
 'HMBS',
 'SHSL1',
 'HSTPT',
 '5DOAN',
 'UHGADA',
 'PSD160',
 'PRAMPC',
 'LPADSS',
 'UDCPDP',
 'ATPPRT',
 'DM_5drib_c',
 'MPTG',
 'KDOCT2',
 'ALAR',
 'UAGPT3',
 'DASYN160',
 'METAT',
 'KDOPS',
 'NNATr',
 'NDPK4',
 'PSSA160',
 'UAGAAT',
 'CU2tpp',
 'UAGDP',
 'MTHFR2',
 'ACCOAC',
 'U23GAAT',
 'PRAIi',
 'MOAT2',
 'GLU5K',
 'ALAALAr',
 'ACPPAT161',
 'TDSK',
 'PAPPT3',
 'METS',
 'HSST',
 'DTMPK',
 'APG3PAT161',
 'GF6PTA',
 'COBALT2tpp',
 'APG3PAT160',
 'UAPGR',
 'A5PISO',
 'AGPAT161',
 'CYSTL',
 'MOAT',
 'GLUTRR',
 'PSSA161',
 'PMPK',
 'TMPK',
 'KDOPP',
 'NADK',
 'PE161abcpp',
 'MCTP1App',
 'UPP3S',
 'IGPDH',
 'K2L4Aabctex',
 'UAMAGS',
 'G1PACT',
 'DHFR',
 'HISTD',
 'UAGCVT',
 'ACPPAT160',
 'NADS1',
 'IGPS',
 'CTPS2',
 'MCOATA',
 'G5SD',
 'PPNDH',
 'GLUTRS',
 'PSD161',
 'G1SAT'

In [275]:
union = pd.Series(np.union1d(w, w2))
unclassified_above_zero = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_above_zero)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_above_zero)
set3= set1-set2
w2= list(set3)

unclassified_above_zero= list(unclassified_above_zero)
unclassified_above_zero


[]

In [276]:
set1= set(inANDdecr_above_zero)
set2= set(w)
set3= set(unclassified_above_zero)
set4= set1-set2-set3
inANDdecr_above_zero= list(set4)

mon_decr_above_zero= list(mon_decr_above_zero) + list(w)


set1= set(inANDdecr_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_above_zero= list(set3)

mon_incr_above_zero= list(mon_incr_above_zero) + list(w2)


In [277]:
i2= 0
mon_incr_below_zero= list()
mon_decr_below_zero= list()

if len(diff_indecr_below_zero)> 0:
    
    for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero)
set2= set(mon_decr_below_zero)
set3= set(mon_incr_below_zero)
inANDdecr_below_zero= list(set1-set2-set3)
inANDdecr_below_zero


['EX_orot_e',
 'TYRTA',
 'IPPMIb',
 'EX_nh4_e',
 'PHETA1',
 'GLUNpp',
 'G3PD2',
 'IMPC',
 'EX_metsox_R__L_e',
 'DHORTS',
 'IPPMIa',
 'EX_met__L_e',
 'ASPTA',
 'EX_pi_e',
 'AIRC3',
 'ASAD',
 'SDPTA',
 'ACOTA',
 'EX_mso3_e',
 'EX_sulfac_e',
 'EX_isetac_e',
 'EX_taur_e',
 'EX_metsox_S__L_e',
 'EX_26dap__M_e',
 'GLCptspp',
 'ORPT',
 'ILETA',
 'HSDy',
 'EX_skm_e',
 'PGAMT',
 'GLUR',
 'AGPR',
 'KARA1',
 'EX_so4_e']

In [278]:
w= list()
w2= list()
i2=0

if len(inANDdecr_below_zero)> 0:
    
    while i2 < len(inANDdecr_below_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


[]

In [279]:
union = pd.Series(np.union1d(w, w2))
unclassified_below_zero = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_below_zero)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_below_zero)
set3= set1-set2
w2= list(set3)

unclassified_below_zero= list(unclassified_below_zero)
unclassified_below_zero


[]

In [280]:
set1= set(inANDdecr_below_zero)
set2= set(w)
set3= set(unclassified_below_zero)
set4= set1-set2-set3
inANDdecr_below_zero= list(set4)

mon_decr_below_zero= list(mon_decr_below_zero) + list(w)


set1= set(inANDdecr_below_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_below_zero= list(set3)

mon_incr_below_zero= list(mon_incr_below_zero) + list(w2)


In [281]:
i2= 0
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

if len(indecr_and_stable_above_zero)> 0:
    
    for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


[]

In [282]:
w= list()
w2= list()
i2=0

if len(inANDdecr_and_stable_above_zero)> 0:
    
    while i2 < len(inANDdecr_and_stable_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


[]

In [283]:
union = pd.Series(np.union1d(w, w2))
unclassified_above_zero2 = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_above_zero2)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_above_zero2)
set3= set1-set2
w2= list(set3)

unclassified_above_zero= list(unclassified_above_zero2)+list(unclassified_above_zero)
unclassified_above_zero


[]

In [284]:
set1= set(inANDdecr_and_stable_above_zero)
set2= set(w)
set3= set(unclassified_above_zero2)
set4= set1-set2-set3
inANDdecr_and_stable_above_zero= list(set4)

mon_decr_and_stable_above_zero= list(mon_decr_and_stable_above_zero) + list(w)


set1= set(inANDdecr_and_stable_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_incr_and_stable_above_zero= list(mon_incr_and_stable_above_zero) + list(w2)


In [285]:
i2= 0
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

if len(indecr_and_stable_below_zero)> 0:
    
    for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero.append(r_id)

        i2= i2 + 1

set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


[]

In [286]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero):       #this code finds reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


[]

In [287]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero= list(unclassified_below_zero2)+list(unclassified_below_zero)
unclassified_below_zero


[]

In [288]:
set1= set(inANDdecr_and_stable_below_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero2)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero= list(set5)

mon_decr_and_stable_below_zero= list(mon_decr_and_stable_below_zero) + list(w3)
mon_incr_and_stable_below_zero= list(mon_incr_and_stable_below_zero) + list(w4)


In [289]:
i= 0
over_below_zero_and_under_above_zero= list()
under_below_zero_and_over_above_zero= list()
over_below_zero_and_over_above_zero= list()
under_below_zero_and_under_above_zero= list()
r_id = a.reactions[0]

for r_id in a.reactions:    #separates reactions based on whether they have less than 30 or not values both in the above and the below zero interval
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

    while n < int(((len(csvFile)-1)/2)+1):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(4)
    
    
    n= int((len(csvFile)-1)/2)
    l4= list()
    
    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(4)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2)>30 or len(csvFile2)==30) and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
        
        over_below_zero_and_under_above_zero.append(r_id)
        
    if len(csvFile2)<30 and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        under_below_zero_and_over_above_zero.append(r_id)
    
    if (len(csvFile2)>30 or len(csvFile2)==30) and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        over_below_zero_and_over_above_zero.append(r_id)
        
    if len(csvFile2)<30 and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
    
        under_below_zero_and_under_above_zero.append(r_id)
    
    
    i= i+1
    
over_below_zero_and_under_above_zero


['EX_enter_e',
 'EX_feenter_e',
 'CLIPAabctex',
 'CPPPGO',
 'ECOAH3',
 'ECOAH4',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'LIPAabctex',
 'RPI',
 'PPPGO',
 'PPPGO3']

In [290]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_diff_indecr_above_zero= list()
rest_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
flat_below_zero_and_one_to_three_data_point_above_zero= list()
diff_indecr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(over_below_zero_and_under_above_zero)> 0:

    for r_id in over_below_zero_and_under_above_zero:    #separates over_below_zero_and_under_above_zero reactions in more specific subcategories
    
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(4)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.0001 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.0001 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.0001:
                count2= count2+1
    
            n2= n2+1
    
        if count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and len(csvFile3)<4:
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> 2 and count2< len(csvFile3)-2):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> 2 and len(csvFile3) < 6):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 3:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and len(csvFile3)<4:
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and count2< len(csvFile3)-2):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
        elif count< 0.05*len(csvFile2) and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 2 and count2< len(csvFile3)-2):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 3:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif count< 0.05*len(csvFile2) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2> 2 and len(csvFile3) < 6):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [291]:
i=0
one_to_three_data_point_below_zero_and_diff_indecr_above_zero= list()
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_flat_above_zero= list()

if len(under_below_zero_and_over_above_zero)> 0:

    for r_id in under_below_zero_and_over_above_zero:    #separates under_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(4)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.0001 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.0001 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.0001:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-2) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 3 and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-2) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<3 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

            
    
        elif len(csvFile2)<4 and count2< 0.05*len(csvFile3):
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-2) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [292]:
i=0

if len(over_below_zero_and_over_above_zero)> 0:

    for r_id in over_below_zero_and_over_above_zero:    #separates over_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(4)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.0001 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.0001 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.0001:
                count2= count2+1
    
            n2= n2+1
    
        if (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count<0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['TALA']

In [293]:
i=0
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero= list()

if len(under_below_zero_and_under_above_zero)> 0:
    
    for r_id in under_below_zero_and_under_above_zero:    #separates under_below_zero_and_under_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(4)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.0001 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.0001:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.0001 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.0001:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and len(csvFile3)<4:
            one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

        elif len(csvFile2)<4 and (count2> 2 and count2< len(csvFile3)-2):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif (count> 2 and count< len(csvFile2)-2) and len(csvFile3)<4:       
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> 2 and len(csvFile2) < 6) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> 1 and len(csvFile2) < 3) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
            
    
        elif (count> 2 and count< len(csvFile2)-2) and (count2> 2 and count2< len(csvFile3)-2):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 2 and count2< len(csvFile3)-2):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 2 and count2< len(csvFile3)-2):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and count2< len(csvFile3)-2):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 3 and (count2> 2 and count2< len(csvFile3)-2):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 2 and len(csvFile3) < 6):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 1 and len(csvFile3) < 3):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-2) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-2) and (count2> 2 and len(csvFile3) < 6):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-2) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> 2 and len(csvFile3) < 6):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)

        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and count2< 3:
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and count2< 3:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and count2< 3:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['TALA']

In [294]:
flat_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero
indecr_and_stable_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_diff_indecr_above_zero
diff_indecr_below_zero_and_indecr_and_stable_above_zero
diff_indecr_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_diff_indecr_above_zero
flat_below_zero_and_indecr_and_stable_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_flat_above_zero


indecr_and_stable_below_zero_and_indecr_and_stable_above_zero
indecr_and_stable_below_zero_and_flat_above_zero
one_to_three_data_point_above_zero
one_to_three_data_point_below_zero


['EX_k_e',
 'PAPA120pp',
 'PAPA140pp',
 'PAPA141pp',
 'PAPA160pp',
 'PAPA180pp',
 'UDCPDPpp']

In [295]:
len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+len(indecr_and_stable_below_zero_and_flat_above_zero)+len(one)+len(zero)+len(flat_below_zero_and_flat_above_zero)+len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_flat_above_zero)+len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_diff_indecr_above_zero)+len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_diff_indecr_above_zero)+len(flat_below_zero_and_indecr_and_stable_above_zero)+len(flat_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_flat_above_zero)+len(one_at_zero)+len(indecr_and_stable_above_zero)+len(indecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(diff_indecr_above_zero)+len(diff_indecr_below_zero)+len(one_to_three_data_point_above_zero)+len(one_to_three_data_point_below_zero)


2726

In [296]:
lt= list(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+list(indecr_and_stable_below_zero_and_flat_above_zero)+list(one)+list(zero)+list(flat_below_zero_and_flat_above_zero)+list(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_flat_above_zero)+list(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_diff_indecr_above_zero)+list(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+list(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_diff_indecr_above_zero)+list(flat_below_zero_and_indecr_and_stable_above_zero)+list(flat_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_flat_above_zero)+list(one_at_zero)+list(indecr_and_stable_above_zero)+list(indecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(diff_indecr_above_zero)+list(diff_indecr_below_zero)+list(one_to_three_data_point_above_zero)+list(one_to_three_data_point_below_zero)


In [297]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [298]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [299]:
i2= 0
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

if len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero:                    #this code finds which of the indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero

set1= set(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set3= set(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero


[]

In [300]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
    
        i2= i2+1

w3


[]

In [301]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_one_to_three_data_point_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_one_to_three_data_point_above_zero= list(unclassified_below_zero_and_one_to_three_data_point_above_zero)
unclassified_below_zero_and_one_to_three_data_point_above_zero


[]

In [302]:
set1= set(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set5)

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [303]:
i2= 0
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                  #this code finds which of the indecr_and_stable_below_zero_and_diff_indecr_above_zero reactions have an increasing trend and which a decreasing trend above zero
    
        r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [304]:
w3= list()
w4= list()
i2=0

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(indecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable above zero
    
        i= int((len(csvFile)-1)/2)
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        
        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


[]

In [305]:
union = pd.Series(np.union1d(w3, w4))
indecr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

indecr_and_stable_below_zero_and_unclassified_above_zero= list(indecr_and_stable_below_zero_and_unclassified_above_zero)
indecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [306]:
set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set5)

indecr_and_stable_below_zero_and_mon_decr_above_zero= list(indecr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
indecr_and_stable_below_zero_and_mon_incr_above_zero= list(indecr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [307]:
i2= 0
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
incr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                #this code finds which of the indecr_and_stable_below_zero_and_mon_decr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['PYK', 'FLDR2']

In [308]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_decr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [309]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_decr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_decr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_decr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_decr_above_zero= list(unclassified_below_zero_and_mon_decr_above_zero)
unclassified_below_zero_and_mon_decr_above_zero


[]

In [310]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_decr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_decr_above_zero= list(decr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero) + list(w4)


In [311]:
i2= 0
decr_and_stable_below_zero_and_mon_incr_above_zero= list()
incr_and_stable_below_zero_and_mon_incr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_incr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_mon_incr_above_zero:              #this code finds which of the indecr_and_stable_below_zero_and_mon_incr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_incr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_incr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero


[]

In [312]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_incr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [313]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_incr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_incr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_incr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_incr_above_zero= list(unclassified_below_zero_and_mon_incr_above_zero)
unclassified_below_zero_and_mon_incr_above_zero


[]

In [314]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_incr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [315]:
i2= 0
decr_and_stable_below_zero_and_inANDdecr_above_zero= list()
incr_and_stable_below_zero_and_inANDdecr_above_zero= list()

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_inANDdecr_above_zero:           #this code finds which of the indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)


        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(decr_and_stable_below_zero_and_inANDdecr_above_zero)
set3= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [316]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [317]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_inANDdecr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_inANDdecr_above_zero= list(unclassified_below_zero_and_inANDdecr_above_zero)
unclassified_below_zero_and_inANDdecr_above_zero


[]

In [318]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_inANDdecr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [319]:
i2= 0
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list()

if len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)
set2= set(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)
set3= set(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [320]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [321]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [322]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero) + list(w4)


In [323]:
i2= 0
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

if len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_diff_indecr_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)
set2= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
set3= set(one_to_three_data_point_below_zero_and_mon_incr_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero


[]

In [324]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [325]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [326]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set5)

one_to_three_data_point_below_zero_and_mon_decr_above_zero= list(one_to_three_data_point_below_zero_and_mon_decr_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list(one_to_three_data_point_below_zero_and_mon_incr_above_zero) + list(w4)


In [327]:
i2= 0
diff_indecr_below_zero_and_mon_incr_above_zero= list()
diff_indecr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_diff_indecr_above_zero:
    
        r_id = diff_indecr_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                diff_indecr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                diff_indecr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_diff_indecr_above_zero)
set2= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set3= set(diff_indecr_below_zero_and_mon_incr_above_zero)
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
diff_indecr_below_zero_and_inANDdecr_above_zero


['GHMT2r',
 'PPC',
 'ASNS2',
 'ENO',
 'FACOAE120',
 'ECOAH2',
 'CYTK1',
 'GARFT',
 'EX_co2_e',
 'ECOAH6',
 'FADRx',
 'CBMKr',
 'ECOAH7',
 'AACPS4',
 'AACPS7',
 'FACOAE161',
 'FUM',
 'ECOAH5',
 'GND',
 'NDPK3',
 'PIt2rpp',
 'Kt2pp',
 'SO4t2pp',
 'RPE',
 'TKT2',
 'IMPD',
 'ECOAH4',
 'FACOAE160',
 'MALS',
 'MTHFC',
 'ECOAH3',
 'MTHFD',
 'POR5',
 'ICL',
 'TPI',
 'TKT1',
 'GAPD',
 'AACPS3',
 'AKGDH',
 'ECOAH1',
 'MDH']

In [328]:
w3= list()
w4= list()
i2=0

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(diff_indecr_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['FACOAE120',
 'CYTK1',
 'ECOAH6',
 'ECOAH7',
 'AACPS4',
 'AACPS7',
 'FACOAE161',
 'NDPK3',
 'Kt2pp',
 'SO4t2pp',
 'IMPD',
 'FACOAE160',
 'MALS',
 'POR5',
 'ICL',
 'AACPS3']

In [329]:
union = pd.Series(np.union1d(w3, w4))
diff_indecr_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(diff_indecr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(diff_indecr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

diff_indecr_below_zero_and_unclassified_above_zero= list(diff_indecr_below_zero_and_unclassified_above_zero)
diff_indecr_below_zero_and_unclassified_above_zero


['ECOAH7']

In [330]:
set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(diff_indecr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set5)

diff_indecr_below_zero_and_mon_decr_above_zero= list(diff_indecr_below_zero_and_mon_decr_above_zero) + list(w3)
diff_indecr_below_zero_and_mon_incr_above_zero= list(diff_indecr_below_zero_and_mon_incr_above_zero) + list(w4)


In [331]:
i2= 0
mon_decr_below_zero_and_unclassified_above_zero= list()
mon_incr_below_zero_and_unclassified_above_zero= list()

if len(diff_indecr_below_zero_and_unclassified_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_unclassified_above_zero:
    
        r_id = diff_indecr_below_zero_and_unclassified_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_unclassified_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_unclassified_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_unclassified_above_zero)
set2= set(mon_decr_below_zero_and_unclassified_above_zero)
set3= set(mon_incr_below_zero_and_unclassified_above_zero)
inANDdecr_below_zero_and_unclassified_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_unclassified_above_zero


[]

In [332]:
i2= 0
mon_decr_below_zero_and_mon_decr_above_zero= list()
mon_incr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_decr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_above_zero)
set3= set(mon_incr_below_zero_and_mon_decr_above_zero)
inANDdecr_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_decr_above_zero


['CYTD',
 'RNDR3b',
 'PGM',
 'CO2tpp',
 'RNDR2b',
 'ACOAD4f',
 'APH161',
 'P5CD',
 'THD2pp',
 'RNDR4',
 'PPS',
 'MGSA',
 'EX_val__L_e',
 'PGK',
 'FBP',
 'ACOAD1f',
 'GMPR',
 'RNDR4b',
 'ACOAD2f',
 'RPI',
 'PGI',
 'GLUDy',
 'RNDR1b',
 'ACOAD6f',
 'ACOAD3f',
 'CYSDS',
 'GLYCDx',
 'GUAD',
 'ME2',
 'CO2tex',
 'RNDR1',
 'OBTFL',
 'PROD2',
 'ACOAD5f',
 'FTHFD',
 'EX_indole_e',
 'RNDR3',
 'NACODA',
 'PPCK',
 'CITL',
 'G6PDA',
 'EX_ura_e',
 'EX_quin_e',
 'MACPD',
 'RNDR2']

In [333]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_decr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

        
        
w3


[]

In [334]:
w4

['CYTD',
 'APH161',
 'P5CD',
 'MGSA',
 'GMPR',
 'ACOAD6f',
 'CYSDS',
 'GUAD',
 'PROD2',
 'FTHFD',
 'EX_indole_e',
 'G6PDA',
 'MACPD']

In [335]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_decr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_decr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_decr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_decr_above_zero= list(unclassified_below_zero_and_mon_decr_above_zero)+list(unclassified_below_zero_and_mon_decr_above_zero2)
unclassified_below_zero_and_mon_decr_above_zero


[]

In [336]:
w4

set1= set(inANDdecr_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_decr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_mon_decr_above_zero= list(set5)

mon_decr_below_zero_and_mon_decr_above_zero= list(mon_decr_below_zero_and_mon_decr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_decr_above_zero= list(mon_incr_below_zero_and_mon_decr_above_zero) + list(w4)


In [337]:
i2= 0
mon_decr_below_zero_and_mon_incr_above_zero= list()
mon_incr_below_zero_and_mon_incr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_incr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_incr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_incr_above_zero)
set2= set(mon_decr_below_zero_and_mon_incr_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_above_zero)
inANDdecr_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_incr_above_zero


[]

In [338]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_incr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [339]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_incr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_incr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_incr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_incr_above_zero= list(unclassified_below_zero_and_mon_incr_above_zero)+list(unclassified_below_zero_and_mon_incr_above_zero2)
unclassified_below_zero_and_mon_incr_above_zero


[]

In [340]:
w4

set1= set(inANDdecr_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_incr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_mon_incr_above_zero= list(set5)

mon_decr_below_zero_and_mon_incr_above_zero= list(mon_decr_below_zero_and_mon_incr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_above_zero= list(mon_incr_below_zero_and_mon_incr_above_zero) + list(w4)


In [341]:
i2= 0
mon_decr_below_zero_and_inANDdecr_above_zero= list()
mon_incr_below_zero_and_inANDdecr_above_zero= list()

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_inANDdecr_above_zero:                  
        
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_inANDdecr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(mon_decr_below_zero_and_inANDdecr_above_zero)
set3= set(mon_incr_below_zero_and_inANDdecr_above_zero)
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_above_zero


[]

In [342]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [343]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_inANDdecr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_inANDdecr_above_zero= list(unclassified_below_zero_and_inANDdecr_above_zero)+list(unclassified_below_zero_and_inANDdecr_above_zero2)
unclassified_below_zero_and_inANDdecr_above_zero


[]

In [344]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_inANDdecr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set5)

mon_decr_below_zero_and_inANDdecr_above_zero= list(mon_decr_below_zero_and_inANDdecr_above_zero) + list(w3)
mon_incr_below_zero_and_inANDdecr_above_zero= list(mon_incr_below_zero_and_inANDdecr_above_zero) + list(w4)


In [345]:
i2= 0
mon_decr_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_below_zero_and_indecr_and_stable_above_zero= list()

if len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = diff_indecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_indecr_and_stable_above_zero


[]

In [346]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [347]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_indecr_and_stable_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_indecr_and_stable_above_zero= list(unclassified_below_zero_and_indecr_and_stable_above_zero)
unclassified_below_zero_and_indecr_and_stable_above_zero


[]

In [348]:
w4

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set5)

mon_decr_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [349]:
i2= 0
inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [350]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [351]:
union = pd.Series(np.union1d(w3, w4))
inANDdecr_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(inANDdecr_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(inANDdecr_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

inANDdecr_below_zero_and_unclassified_above_zero= list(inANDdecr_below_zero_and_unclassified_above_zero)+list(inANDdecr_below_zero_and_unclassified_above_zero2)
inANDdecr_below_zero_and_unclassified_above_zero


[]

In [352]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(inANDdecr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [353]:
i2= 0
mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [354]:
w3= list()
w4= list()
i2=0

if len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [355]:
union = pd.Series(np.union1d(w3, w4))
mon_decr_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_decr_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_decr_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

mon_decr_below_zero_and_unclassified_above_zero= list(mon_decr_below_zero_and_unclassified_above_zero)+list(mon_decr_below_zero_and_unclassified_above_zero2)
mon_decr_below_zero_and_unclassified_above_zero


[]

In [356]:
w4

set1= set(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_decr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [357]:
i2= 0
mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [358]:
w3= list()
w4= list()
i2=0

if len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [359]:
union = pd.Series(np.union1d(w3, w4))
mon_incr_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_incr_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_incr_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

mon_incr_below_zero_and_unclassified_above_zero= list(mon_incr_below_zero_and_unclassified_above_zero)+list(mon_incr_below_zero_and_unclassified_above_zero2)
mon_incr_below_zero_and_unclassified_above_zero


['ECOAH7']

In [360]:
w4

set1= set(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_incr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [361]:
i2= 0
mon_decr_below_zero_and_one_to_three_data_point_above_zero= list()
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_one_to_three_data_point_above_zero:
    
        r_id = diff_indecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(mon_decr_below_zero_and_one_to_three_data_point_above_zero)
set3= set(mon_incr_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero


[]

In [362]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [363]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_one_to_three_data_point_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_one_to_three_data_point_above_zero= list(unclassified_below_zero_and_one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
unclassified_below_zero_and_one_to_three_data_point_above_zero


[]

In [364]:
w4

set1= set(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set5)

mon_decr_below_zero_and_one_to_three_data_point_above_zero= list(mon_decr_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list(mon_incr_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [365]:
i2= 0
flat_below_zero_and_mon_decr_above_zero= list()
flat_below_zero_and_mon_incr_above_zero= list()

if len(flat_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in flat_below_zero_and_diff_indecr_above_zero:
    
        r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [366]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [367]:
union = pd.Series(np.union1d(w3, w4))
flat_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(flat_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(flat_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

flat_below_zero_and_unclassified_above_zero= list(flat_below_zero_and_unclassified_above_zero)
flat_below_zero_and_unclassified_above_zero


[]

In [368]:
w4

set1= set(flat_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(flat_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
flat_below_zero_and_inANDdecr_above_zero= list(set5)

flat_below_zero_and_mon_decr_above_zero= list(flat_below_zero_and_mon_decr_above_zero) + list(w3)
flat_below_zero_and_mon_incr_above_zero= list(flat_below_zero_and_mon_incr_above_zero) + list(w4)


In [369]:
i2= 0
flat_below_zero_and_mon_decr_and_stable_above_zero= list()
flat_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(flat_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in flat_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_mon_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [370]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [371]:
union = pd.Series(np.union1d(w3, w4))
flat_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(flat_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(flat_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

flat_below_zero_and_unclassified_above_zero= list(flat_below_zero_and_unclassified_above_zero)+list(flat_below_zero_and_unclassified_above_zero2)
flat_below_zero_and_unclassified_above_zero


[]

In [372]:
w4

set1= set(flat_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(flat_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

flat_below_zero_and_mon_decr_and_stable_above_zero= list(flat_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
flat_below_zero_and_mon_incr_and_stable_above_zero= list(flat_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [373]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [374]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
        i2= i2+1

w3


[]

In [375]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)
unclassified_below_zero_and_flat_above_zero


[]

In [376]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_flat_above_zero= list(set5)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [377]:
i2= 0
mon_decr_and_stable_below_zero_and_flat_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

if len(indecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_flat_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_flat_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


[]

In [378]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [379]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)+list(unclassified_below_zero_and_flat_above_zero2)
unclassified_below_zero_and_flat_above_zero


[]

In [380]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_flat_above_zero= list(mon_decr_and_stable_below_zero_and_flat_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_flat_above_zero= list(mon_incr_and_stable_below_zero_and_flat_above_zero) + list(w4)


In [381]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [382]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [383]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)+list(unclassified_below_zero_and_flat_above_zero2)
unclassified_below_zero_and_flat_above_zero


[]

In [384]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_flat_above_zero= list(set5)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [385]:
i2= 0
mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()

if len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [386]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [387]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_indecr_and_stable_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_indecr_and_stable_above_zero= list(unclassified_below_zero_and_indecr_and_stable_above_zero)+list(unclassified_below_zero_and_indecr_and_stable_above_zero2)
unclassified_below_zero_and_indecr_and_stable_above_zero


[]

In [388]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [389]:
i2= 0
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [390]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [391]:
union = pd.Series(np.union1d(w3, w4))
inANDdecr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [392]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [393]:
i2= 0
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [394]:
w3= list()
w4= list()
i2=0

if len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [395]:
union = pd.Series(np.union1d(w3, w4))
mon_decr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_decr_and_stable_below_zero_and_unclassified_above_zero= list(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
mon_decr_and_stable_below_zero_and_unclassified_above_zero


[]

In [396]:
w4

set1= set(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [397]:
i2= 0
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [398]:
w3= list()
w4= list()
i2=0

if len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [399]:
union = pd.Series(np.union1d(w3, w4))
mon_incr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_incr_and_stable_below_zero_and_unclassified_above_zero= list(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
mon_incr_and_stable_below_zero_and_unclassified_above_zero


[]

In [400]:
w4

set1= set(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [401]:
i2= 0
mon_decr_and_stable_below_zero_and_unclassified_above_zero= list()
mon_incr_and_stable_below_zero_and_unclassified_above_zero= list()

if len(indecr_and_stable_below_zero_and_unclassified_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_unclassified_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_unclassified_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_unclassified_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_unclassified_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [402]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_unclassified_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [403]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_unclassified_above_zero= list(unclassified_below_zero_and_unclassified_above_zero)
unclassified_below_zero_and_unclassified_above_zero


[]

In [404]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_unclassified_above_zero= list(mon_decr_and_stable_below_zero_and_unclassified_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_unclassified_above_zero= list(mon_incr_and_stable_below_zero_and_unclassified_above_zero) + list(w4)


In [405]:
i2= 0
unclassified_below_zero_and_mon_decr_and_stable_above_zero= list()
unclassified_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(unclassified_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in unclassified_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = unclassified_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                unclassified_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                unclassified_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set2= set(unclassified_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(unclassified_below_zero_and_mon_incr_and_stable_above_zero)
unclassified_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
unclassified_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [406]:
w3= list()
w4= list()
i2=0

if len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = unclassified_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_sorbitol.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(4)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [407]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_unclassified_above_zero= list(unclassified_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_unclassified_above_zero2)
unclassified_below_zero_and_unclassified_above_zero


[]

In [408]:
w4

set1= set(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
unclassified_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

unclassified_below_zero_and_mon_decr_and_stable_above_zero= list(unclassified_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
unclassified_below_zero_and_mon_incr_and_stable_above_zero= list(unclassified_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [409]:
d= pd.DataFrame (one, columns = ['Reaction names'])
d.to_csv("FINAL one feasible value NOT at zero reactions -30 to 30 4 decimals sorbitol.csv")
d1= pd.DataFrame (zero, columns = ['Reaction names'])
d1.to_csv("FINAL zero reactions -30 to 30 4 decimals sorbitol.csv")
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D.to_csv("FINAL one feasible value at zero reactions -30 to 30 4 decimals sorbitol.csv")
D1= pd.DataFrame (flat_above_zero, columns = ['Reaction names'])
D1.to_csv("FINAL flat above zero reactions -30 to 30 4 decimals sorbitol.csv")
D2= pd.DataFrame (flat_below_zero, columns = ['Reaction names'])
D2.to_csv("FINAL flat below zero reactions -30 to 30 4 decimals sorbitol.csv")
D3= pd.DataFrame (mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D3.to_csv("FINAL mon_decr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D4= pd.DataFrame (mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D4.to_csv("FINAL mon_incr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D5= pd.DataFrame (inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D5.to_csv("FINAL inANDdecr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D6= pd.DataFrame (mon_decr_and_stable_below_zero, columns = ['Reaction names'])
D6.to_csv("FINAL increasing and stable below zero reactions -30 to 30 4 decimals sorbitol.csv")
D7= pd.DataFrame (mon_incr_and_stable_below_zero, columns = ['Reaction names'])
D7.to_csv("FINAL decreasing and stable below reactions -30 to 30 4 decimals sorbitol.csv")
D8= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D8.to_csv("FINAL inANDdecr_and_stable_below_zero reactions -30 to 30 4 decimals sorbitol.csv")
D9= pd.DataFrame (decr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D9.to_csv("FINAL increasing and stable below zero and one to three data point above zero reactions -30 to 30 4 decimals sorbitol.csv")
D10= pd.DataFrame (incr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D10.to_csv("FINAL decreasing and stable below zero and one to three data point above zero reactions -30 to 30 4 decimals sorbitol.csv")
D11= pd.DataFrame (inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D11.to_csv("FINAL inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D12= pd.DataFrame (decr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D12.to_csv("FINAL increasing and stable below zero and decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D13= pd.DataFrame (incr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D13.to_csv("FINAL decreasing and stable below zero and decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D14= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D14.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_decr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D15= pd.DataFrame (decr_and_stable_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D15.to_csv("FINAL increasing and stable below zero and increasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D16= pd.DataFrame (incr_and_stable_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D16.to_csv("FINAL decreasing and stable below zero and increasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D17= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D17.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_incr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D18= pd.DataFrame (decr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D18.to_csv("FINAL increasing and stable below zero and increasing and decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D19= pd.DataFrame (incr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D19.to_csv("FINAL decreasing and stable below zero and increasing and decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D20= pd.DataFrame (inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D20.to_csv("FINAL inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D21= pd.DataFrame (one_to_three_data_point_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D21.to_csv("FINAL one_to_three_data_point_below_zero_and_decr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D22= pd.DataFrame (one_to_three_data_point_below_zero_and_incr_and_stable_above_zero, columns = ['Reaction names'])
D22.to_csv("FINAL one_to_three_data_point_below_zero_and_incr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D23= pd.DataFrame (one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D23.to_csv("FINAL one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D24= pd.DataFrame (one_to_three_data_point_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D24.to_csv("FINAL one_to_three_data_point_below_zero_and_mon_decr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D25= pd.DataFrame (one_to_three_data_point_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D25.to_csv("FINAL one_to_three_data_point_below_zero_and_mon_incr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D26= pd.DataFrame (one_to_three_data_point_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D26.to_csv("FINAL one_to_three_data_point_below_zero_and_inANDdecr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D27= pd.DataFrame (inANDdecr_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D27.to_csv("FINAL inANDdecr_below_zero_and_mon_decr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D28= pd.DataFrame (mon_decr_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D28.to_csv("FINAL increasing below zero decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D29= pd.DataFrame (mon_incr_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D29.to_csv("FINAL decreasing below zero decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D30= pd.DataFrame (inANDdecr_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D30.to_csv("FINAL inANDdecr_below_zero_and_mon_incr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D31= pd.DataFrame (mon_decr_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D31.to_csv("FINAL increasing below zero and increasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D32= pd.DataFrame (mon_incr_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D32.to_csv("FINAL decreasing below zero and increasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D33= pd.DataFrame (inANDdecr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D33.to_csv("FINAL inANDdecr_below_zero_and_inANDdecr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D34= pd.DataFrame (mon_decr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D34.to_csv("FINAL increasing below zero and increasing and decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D35= pd.DataFrame (mon_incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D35.to_csv("FINAL decreasing below zero and increasing and decreasing above zero reactions -30 to 30 4 decimals sorbitol.csv")
D36= pd.DataFrame (inANDdecr_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D36.to_csv("FINAL inANDdecr_below_zero_and_mon_decr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D37= pd.DataFrame (inANDdecr_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D37.to_csv("FINAL inANDdecr_below_zero_and_mon_incr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D38= pd.DataFrame (inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D38.to_csv("FINAL inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D39= pd.DataFrame (mon_decr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D39.to_csv("FINAL increasing below zero and increasing and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D40= pd.DataFrame (mon_decr_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D40.to_csv("FINAL increasing below zero and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D41= pd.DataFrame (mon_decr_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D41.to_csv("FINAL increasing below zero and increasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D42= pd.DataFrame (mon_incr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D42.to_csv("FINAL decreasing below zero and increasing and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D43= pd.DataFrame (mon_incr_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D43.to_csv("FINAL decreasing below zero and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D44= pd.DataFrame (mon_incr_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D44.to_csv("FINAL decreasing below zero and increasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D45= pd.DataFrame (inANDdecr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D45.to_csv("FINAL inANDdecr_below_zero_and_one_to_three_data_point_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D46= pd.DataFrame (mon_decr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D46.to_csv("FINAL increasing below zero and one to three data point above zero reactions -30 to 30 4 decimals sorbitol.csv")
D47= pd.DataFrame (mon_incr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D47.to_csv("FINAL decreasing below zero and one to three data point above zero reactions -30 to 30 4 decimals sorbitol.csv")
D48= pd.DataFrame (flat_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D48.to_csv("FINAL flat_below_zero_and_inANDdecr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D49= pd.DataFrame (flat_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D49.to_csv("FINAL flat_below_zero_and_mon_decr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D50= pd.DataFrame (flat_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D50.to_csv("FINAL flat_below_zero_and_mon_incr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D51= pd.DataFrame (flat_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D51.to_csv("FINAL flat_below_zero_and_inANDdecr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D52= pd.DataFrame (flat_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D52.to_csv("FINAL flat_below_zero_and_mon_decr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D53= pd.DataFrame (flat_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D53.to_csv("FINAL flat_below_zero_and_mon_incr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D54= pd.DataFrame (inANDdecr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D54.to_csv("FINAL inANDdecr_and_stable_below_zero_and_flat_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D55= pd.DataFrame (mon_decr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D55.to_csv("FINAL increasing and stable below zero and flat above zero reactions -30 to 30 4 decimals sorbitol.csv")
D56= pd.DataFrame (mon_incr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D56.to_csv("FINAL decreasing and stable below zero and flat above zero reactions -30 to 30 4 decimals sorbitol.csv")
D57= pd.DataFrame (inANDdecr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D57.to_csv("FINAL inANDdecr_below_zero_and_flat_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D58= pd.DataFrame (mon_decr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D58.to_csv("FINAL increasing below zero and flat above zero reactions -30 to 30 4 decimals sorbitol.csv")
D59= pd.DataFrame (mon_incr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D59.to_csv("FINAL decreasing below zero and flat above zero reactions -30 to 30 4 decimals sorbitol.csv")
D60= pd.DataFrame (inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D60.to_csv("FINAL inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D61= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D61.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D62= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D62.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D63= pd.DataFrame (mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D63.to_csv("FINAL increasing and stable below zero and increasing and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D64= pd.DataFrame (mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D64.to_csv("FINAL increasing and stable below zero and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D65= pd.DataFrame (mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D65.to_csv("FINAL increasing and stable below zero and increasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D66= pd.DataFrame (mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D66.to_csv("FINAL decreasing and stable below zero and increasing and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D67= pd.DataFrame (mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D67.to_csv("FINAL decreasing and stable below zero and decreasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D68= pd.DataFrame (mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D68.to_csv("FINAL decreasing and stable below zero and increasing and stable above zero reactions -30 to 30 4 decimals sorbitol.csv")
D69= pd.DataFrame (flat_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D69.to_csv("FINAL flat_below_zero_and_flat_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D70= pd.DataFrame (one_to_three_data_point_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D70.to_csv("FINAL one_to_three_data_point_below_zero_and_flat_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D71= pd.DataFrame (one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D71.to_csv("FINAL one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D72= pd.DataFrame (flat_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D72.to_csv("FINAL flat_below_zero_and_one_to_three_data_point_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D73= pd.DataFrame (unclassified_above_zero, columns = ['Reaction names'])
D73.to_csv("FINAL unclassified_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D74= pd.DataFrame (unclassified_below_zero, columns = ['Reaction names'])
D74.to_csv("FINAL unclassified_below_zero reactions -30 to 30 4 decimals sorbitol.csv")
D75= pd.DataFrame (one_to_three_data_point_above_zero, columns = ['Reaction names'])
D75.to_csv("FINAL one_to_three_data_point_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D76= pd.DataFrame (one_to_three_data_point_below_zero, columns = ['Reaction names'])
D76.to_csv("FINAL one_to_three_data_point_below_zero reactions -30 to 30 4 decimals sorbitol.csv")
D77= pd.DataFrame (mon_decr_above_zero, columns = ['Reaction names'])
D77.to_csv("FINAL mon_decr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D78= pd.DataFrame (mon_incr_above_zero, columns = ['Reaction names'])
D78.to_csv("FINAL mon_incr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D79= pd.DataFrame (inANDdecr_above_zero, columns = ['Reaction names'])
D79.to_csv("FINAL inANDdecr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D80= pd.DataFrame (mon_decr_below_zero, columns = ['Reaction names'])
D80.to_csv("FINAL increasing below zero reactions -30 to 30 4 decimals sorbitol.csv")
D81= pd.DataFrame (mon_incr_below_zero, columns = ['Reaction names'])
D81.to_csv("FINAL decreasing below zero reactions -30 to 30 4 decimals sorbitol.csv")
D82= pd.DataFrame (inANDdecr_below_zero, columns = ['Reaction names'])
D82.to_csv("FINAL inANDdecr_below_zero reactions -30 to 30 4 decimals sorbitol.csv")
D83= pd.DataFrame (unclassified_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D83.to_csv("FINAL unclassified_below_zero_and_one_to_three_data_point_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D84= pd.DataFrame (unclassified_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D84.to_csv("FINAL unclassified_below_zero_and_mon_decr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D85= pd.DataFrame (unclassified_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D85.to_csv("FINAL unclassified_below_zero_and_mon_incr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D86= pd.DataFrame (unclassified_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D86.to_csv("FINAL unclassified_below_zero_and_inANDdecr_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D87= pd.DataFrame (one_to_three_data_point_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D87.to_csv("FINAL one_to_three_data_point_below_zero_and_unclassified_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D88= pd.DataFrame (inANDdecr_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D88.to_csv("FINAL inANDdecr_below_zero_and_unclassified_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D89= pd.DataFrame (mon_decr_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D89.to_csv("FINAL increasing below zero and unclassified above zero reactions -30 to 30 4 decimals sorbitol.csv")
D90= pd.DataFrame (mon_incr_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D90.to_csv("FINAL decreasing below zero and unclassified above zero reactions -30 to 30 4 decimals sorbitol.csv")
D91= pd.DataFrame (flat_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D91.to_csv("FINAL flat_below_zero_and_unclassified_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D92= pd.DataFrame (unclassified_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D92.to_csv("FINAL unclassified_below_zero_and_flat_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D93= pd.DataFrame (inANDdecr_and_stable_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D93.to_csv("FINAL inANDdecr_and_stable_below_zero_and_unclassified_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D94= pd.DataFrame (mon_decr_and_stable_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D94.to_csv("FINAL increasing and stable below zero and unclassified above zero reactions -30 to 30 4 decimals sorbitol.csv")
D95= pd.DataFrame (mon_incr_and_stable_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D95.to_csv("FINAL decreasing and stable below zero and unclassified above zero reactions -30 to 30 4 decimals sorbitol.csv")
D96= pd.DataFrame (unclassified_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D96.to_csv("FINAL unclassified_below_zero_and_unclassified_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D97= pd.DataFrame (unclassified_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D97.to_csv("FINAL unclassified_below_zero_and_mon_decr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D98= pd.DataFrame (unclassified_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D98.to_csv("FINAL unclassified_below_zero_and_mon_incr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")
D99= pd.DataFrame (unclassified_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D99.to_csv("FINAL unclassified_below_zero_and_inANDdecr_and_stable_above_zero reactions -30 to 30 4 decimals sorbitol.csv")


In [410]:
inANDdecr_below_zero_and_mon_decr_above_zero

['RNDR3b',
 'PGM',
 'CO2tpp',
 'RNDR2b',
 'ACOAD4f',
 'THD2pp',
 'RNDR4',
 'PPS',
 'FBP',
 'EX_val__L_e',
 'PGK',
 'ACOAD1f',
 'RNDR4b',
 'ACOAD2f',
 'RPI',
 'PGI',
 'GLUDy',
 'RNDR1b',
 'ACOAD3f',
 'GLYCDx',
 'ME2',
 'CO2tex',
 'RNDR1',
 'OBTFL',
 'ACOAD5f',
 'RNDR3',
 'NACODA',
 'PPCK',
 'CITL',
 'EX_ura_e',
 'EX_quin_e',
 'RNDR2']

In [411]:
one
zero
one_at_zero
flat_above_zero
flat_below_zero

mon_decr_and_stable_above_zero
mon_incr_and_stable_above_zero
inANDdecr_and_stable_above_zero

mon_decr_and_stable_below_zero
mon_incr_and_stable_below_zero
inANDdecr_and_stable_below_zero

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero

decr_and_stable_below_zero_and_mon_decr_above_zero
incr_and_stable_below_zero_and_mon_decr_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero

decr_and_stable_below_zero_and_mon_incr_above_zero
incr_and_stable_below_zero_and_mon_incr_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero

decr_and_stable_below_zero_and_inANDdecr_above_zero
incr_and_stable_below_zero_and_inANDdecr_above_zero
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero

one_to_three_data_point_below_zero_and_mon_decr_above_zero
one_to_three_data_point_below_zero_and_mon_incr_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_above_zero
mon_decr_below_zero_and_mon_decr_above_zero
mon_incr_below_zero_and_mon_decr_above_zero

inANDdecr_below_zero_and_mon_incr_above_zero
mon_decr_below_zero_and_mon_incr_above_zero
mon_incr_below_zero_and_mon_incr_above_zero

inANDdecr_below_zero_and_inANDdecr_above_zero
mon_decr_below_zero_and_inANDdecr_above_zero
mon_incr_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero

mon_decr_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_below_zero_and_one_to_three_data_point_above_zero
mon_decr_below_zero_and_one_to_three_data_point_above_zero
mon_incr_below_zero_and_one_to_three_data_point_above_zero

flat_below_zero_and_inANDdecr_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_mon_incr_above_zero

flat_below_zero_and_inANDdecr_and_stable_above_zero
flat_below_zero_and_mon_decr_and_stable_above_zero
flat_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_and_stable_below_zero_and_flat_above_zero
mon_decr_and_stable_below_zero_and_flat_above_zero
mon_incr_and_stable_below_zero_and_flat_above_zero

inANDdecr_below_zero_and_flat_above_zero
mon_decr_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero

inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

flat_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero

unclassified_above_zero
unclassified_below_zero
one_to_three_data_point_above_zero
one_to_three_data_point_below_zero

mon_decr_above_zero
mon_incr_above_zero
inANDdecr_above_zero

mon_decr_below_zero
mon_incr_below_zero
inANDdecr_below_zero

unclassified_below_zero_and_one_to_three_data_point_above_zero
unclassified_below_zero_and_mon_decr_above_zero
unclassified_below_zero_and_mon_incr_above_zero
unclassified_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_unclassified_above_zero
inANDdecr_below_zero_and_unclassified_above_zero
mon_decr_below_zero_and_unclassified_above_zero
mon_incr_below_zero_and_unclassified_above_zero

flat_below_zero_and_unclassified_above_zero
unclassified_below_zero_and_flat_above_zero
inANDdecr_and_stable_below_zero_and_unclassified_above_zero
mon_decr_and_stable_below_zero_and_unclassified_above_zero
mon_incr_and_stable_below_zero_and_unclassified_above_zero
unclassified_below_zero_and_unclassified_above_zero

unclassified_below_zero_and_mon_decr_and_stable_above_zero
unclassified_below_zero_and_mon_incr_and_stable_above_zero
unclassified_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [412]:
mon_incr_below_zero_and_unclassified_above_zero

['ECOAH7']

In [413]:
len(flat_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ len(flat_below_zero_and_one_to_three_data_point_above_zero)+ len(one_at_zero)+len(one)+len(zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)+len(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_above_zero)+len(mon_decr_below_zero_and_mon_decr_above_zero)+len(mon_incr_below_zero_and_mon_decr_above_zero)+len(inANDdecr_below_zero_and_mon_incr_above_zero)+len(mon_decr_below_zero_and_mon_incr_above_zero)+len(mon_incr_below_zero_and_mon_incr_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_above_zero)+len(mon_decr_below_zero_and_inANDdecr_above_zero)+len(mon_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_mon_decr_and_stable_above_zero)+len(flat_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_flat_above_zero)+len(mon_decr_and_stable_below_zero_and_flat_above_zero)+len(mon_incr_and_stable_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(unclassified_above_zero)+len(unclassified_below_zero)+len(one_to_three_data_point_below_zero)+len(one_to_three_data_point_above_zero)+len(unclassified_below_zero_and_one_to_three_data_point_above_zero)+len(unclassified_below_zero_and_mon_decr_above_zero)+len(unclassified_below_zero_and_mon_incr_above_zero)+len(unclassified_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_unclassified_above_zero)+len(inANDdecr_below_zero_and_unclassified_above_zero)+len(mon_decr_below_zero_and_unclassified_above_zero)+len(mon_incr_below_zero_and_unclassified_above_zero)+len(flat_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)+len(mon_decr_and_stable_below_zero_and_unclassified_above_zero)+len(mon_incr_and_stable_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_mon_decr_and_stable_above_zero)+len(unclassified_below_zero_and_mon_incr_and_stable_above_zero)+len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)


2726

In [414]:
lt= list(flat_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ list(flat_below_zero_and_one_to_three_data_point_above_zero)+ list(one_at_zero)+list(one)+list(zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(mon_decr_above_zero)+list(mon_incr_above_zero)+list(inANDdecr_above_zero)+list(mon_decr_below_zero)+list(mon_incr_below_zero)+list(inANDdecr_below_zero)+list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+list(decr_and_stable_below_zero_and_mon_incr_above_zero)+list(incr_and_stable_below_zero_and_mon_incr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+list(decr_and_stable_below_zero_and_inANDdecr_above_zero)+list(incr_and_stable_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_above_zero)+list(mon_decr_below_zero_and_mon_decr_above_zero)+list(mon_incr_below_zero_and_mon_decr_above_zero)+list(inANDdecr_below_zero_and_mon_incr_above_zero)+list(mon_decr_below_zero_and_mon_incr_above_zero)+list(mon_incr_below_zero_and_mon_incr_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_above_zero)+list(mon_decr_below_zero_and_inANDdecr_above_zero)+list(mon_incr_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_inANDdecr_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_mon_incr_above_zero)+list(flat_below_zero_and_inANDdecr_and_stable_above_zero)+list(flat_below_zero_and_mon_decr_and_stable_above_zero)+list(flat_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_flat_above_zero)+list(mon_decr_and_stable_below_zero_and_flat_above_zero)+list(mon_incr_and_stable_below_zero_and_flat_above_zero)+list(inANDdecr_below_zero_and_flat_above_zero)+list(mon_decr_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(unclassified_above_zero)+list(unclassified_below_zero)+list(one_to_three_data_point_below_zero)+list(one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_mon_decr_above_zero)+list(unclassified_below_zero_and_mon_incr_above_zero)+list(unclassified_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(inANDdecr_below_zero_and_unclassified_above_zero)+list(mon_decr_below_zero_and_unclassified_above_zero)+list(mon_incr_below_zero_and_unclassified_above_zero)+list(flat_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_decr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_incr_and_stable_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_mon_decr_and_stable_above_zero)+list(unclassified_below_zero_and_mon_incr_and_stable_above_zero)+list(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)


In [415]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [416]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [747]:
i= 0
csvFile = pd.read_csv('FINAL different reactions glu vs lac 4 decimals -30 to 30.csv')
csvFile["Category in lactose"] = ""
csvFile

while i < len(csvFile):
    if csvFile.iloc[i, 2] in one:
        csvFile.iloc[i, 4] = "one"
    if csvFile.iloc[i, 2] in zero:
        csvFile.iloc[i, 4] = "zero"
    if csvFile.iloc[i, 2] in one_at_zero:
        csvFile.iloc[i, 4] = "one_at_zero"
    if csvFile.iloc[i, 2] in flat_above_zero:
        csvFile.iloc[i, 4] = "flat_above_zero"
    if csvFile.iloc[i, 2] in flat_below_zero:
        csvFile.iloc[i, 4] = "flat_below_zero"
    if csvFile.iloc[i, 2] in mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "mon_incr_and_stable_above_zero"   
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_and_stable_below_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero"
    if csvFile.iloc[i, 2] in mon_incr_and_stable_below_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero"
    if csvFile.iloc[i, 2] in decr_and_stable_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and one to three data point above zero"
    if csvFile.iloc[i, 2] in incr_and_stable_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and one to three data point above zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 2] in decr_and_stable_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and decreasing above zero"
    if csvFile.iloc[i, 2] in incr_and_stable_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and decreasing above zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_mon_decr_above_zero"   
    if csvFile.iloc[i, 2] in decr_and_stable_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and increasing above zero"
    if csvFile.iloc[i, 2] in incr_and_stable_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and increasing above zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 2] in decr_and_stable_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and increasing and decreasing above zero"
    if csvFile.iloc[i, 2] in incr_and_stable_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and increasing and decreasing above zero"   
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_decr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_incr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and decreasing above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and decreasing above zero"   
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and increasing above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and increasing above zero"
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and increasing and decreasing above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and increasing and decreasing above zero"
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_mon_incr_and_stable_above_zero"   
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and increasing and decreasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and increasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and increasing and decreasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and increasing and stable above zero"
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and one to three data point above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and one to three data point above zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_mon_incr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_and_stable_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and flat above zero"
    if csvFile.iloc[i, 2] in mon_incr_and_stable_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and flat above zero"
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and flat above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and flat above zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and inANDdecr_and_stable above zero"
    if csvFile.iloc[i, 2] in mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and increasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and inANDdecr_and_stable above zero"
    if csvFile.iloc[i, 2] in mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 2] in mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and increasing and stable above zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 2] in unclassified_above_zero:
        csvFile.iloc[i, 4] = "unclassified_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero"
    if csvFile.iloc[i, 2] in mon_decr_above_zero:
        csvFile.iloc[i, 4] = "mon_decr_above_zero"
    if csvFile.iloc[i, 2] in mon_incr_above_zero:
        csvFile.iloc[i, 4] = "mon_incr_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero:
        csvFile.iloc[i, 4] = "increasing below zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero:
        csvFile.iloc[i, 4] = "decreasing below zero"  
    if csvFile.iloc[i, 2] in inANDdecr_below_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 2] in one_to_three_data_point_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "one_to_three_data_point_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "increasing below zero and unclassified above zero"
    if csvFile.iloc[i, 2] in mon_incr_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "decreasing below zero and unclassified above zero"
    if csvFile.iloc[i, 2] in flat_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "flat_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 2] in inANDdecr_and_stable_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "inANDdecr_and_stable_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 2] in mon_decr_and_stable_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "increasing and stable below zero and unclassified above zero"
    if csvFile.iloc[i, 2] in mon_incr_and_stable_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "decreasing and stable below zero and unclassified above zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_mon_incr_and_stable_above_zero"
    if csvFile.iloc[i, 2] in unclassified_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 4] = "unclassified_below_zero_and_inANDdecr_and_stable_above_zero"

    i= i+1

csvFile
csvFile.to_csv("FINAL different reactions glu vs lac 4 decimals -30 to 30.csv")
